In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
rows = 7
columns = 10
q_value = 1
reward = 2
list_of_s_and_a = 0
wind_list = [0, 0, 0, 1, 1, 1, 2, 2, 1, 0] #[0, -10, 0, 13, 0, 4, 0, 9, 0, 1,  -2, 0, 0, 0, 0, 10, -2] # original [0, 0, 0, 1, 1, 1, 2, 2, 1, 0]
goal = (3,7)
start_state = (3, 0)
epsilon = 0.1
alpha = 0.5  #0.5
gamma = 1  # 1
treasure = (5, 1)

In [3]:
len(wind_list)

10

In [4]:
def initialize_states(tmp_row, tmp_col):
    tmp_list = []
    for i in [-1, 0 ,1]:
        for k in [-1, 0 , 1]:
            #if ((( np.min( [np.max([i + tmp_row + wind_list[tmp_col], 0]),(rows - 1) ]), np.min([np.max([k + tmp_col , 0 ]), (columns - 1)])) != (tmp_row , tmp_col)) ):            
            if ((( np.min( [np.max([i + tmp_row + wind_list[tmp_col], 0]),(rows - 1) ]), np.min([np.max([k + tmp_col , 0 ]), (columns - 1)])) != (tmp_row , tmp_col)) and 
                (i != 0 or k != 0)):
                tmp_list.append(( np.min( [np.max([i + tmp_row + wind_list[tmp_col], 0]),rows - 1 ]), np.min([np.max([k + tmp_col , 0 ]), columns - 1])))
    unique_list = list(set(tmp_list))
    reward_list = [ -1 if elem != goal else -1 for elem in unique_list]  #different reward can be given to terminal state
    for elem in range(len(reward_list)):
        if unique_list[elem] == treasure and (tmp_col == 0 or tmp_col == 1):
            reward_list[elem] = 25
        
    return [unique_list , [0 for elem in unique_list] , reward_list]
        
    

In [5]:
def reset_rewards():
    #print ('reset rewards')
    for r in range(rows):
        for c in range(columns):
            #print(list_of_states[r][c][2])
            for elem in range(len(list_of_states[r][c][2])):
                list_of_states[r][c][2][elem] = -1

In [6]:
def choose_s_and_a(tmp_row, tmp_col, tmp_epsilon):
    greedy_flag = np.random.choice([True, False], p = (1 - tmp_epsilon, tmp_epsilon))
    if not greedy_flag:
        index = np.random.choice(range(len(list_of_states[tmp_row][tmp_col][q_value])))
    else:
        index = np.argmax(np.array(list_of_states[tmp_row][tmp_col][q_value]))
    return  (list_of_states[tmp_row][tmp_col][list_of_s_and_a][index] , list_of_states[tmp_row][tmp_col][q_value][index], list_of_states[tmp_row][tmp_col][reward][index], index)

In [7]:
def single_episode(tmp_row, tmp_col):
    s_r = tmp_row
    s_c = tmp_col
    while True:
        action_1, tmp_val_1, tmp_reward_1, tmp_index_1 = choose_s_and_a(s_r, s_c, epsilon)
        #action_1, tmp_val_1, tmp_reward_1, tmp_index_1
        action_2, tmp_val_2, tmp_reward_2, tmp_index_2 = choose_s_and_a(action_1[0], action_1[1], 0)
        #list_of_states[action_1[0]][action_1[1]][q_value][tmp_index_1] = tmp_val_1 + alpha * (tmp_reward_1 + gamma * tmp_val_2 - tmp_val_1)
        list_of_states[s_r][s_c][q_value][tmp_index_1] = tmp_val_1 + alpha * (tmp_reward_1 + gamma * tmp_val_2 - tmp_val_1)
        if action_1 != goal:
            s_r = action_1[0]
            s_c = action_1[1]
            if treasure[0] == action_1[0] and treasure[1] == action_1[1]:
                reset_rewards()
        else:
            break
    

In [8]:
def follow_greedy_path(tmp_row, tmp_col):
    results_arr = np.zeros((rows, columns))
    s_r = tmp_row
    s_c = tmp_col
    cnt = 0
    while True:
        action_1, tmp_val_1, tmp_reward_1, tmp_index_1 = choose_s_and_a(s_r, s_c, 0)
        cnt += 1
        results_arr[rows -1  - action_1[0],  action_1[1]] = cnt
        if action_1 != goal:
            s_r = action_1[0]
            s_c = action_1[1]
            if cnt > rows * columns :
                return (-1, results_arr)

        else:
            results_arr[rows -1 - goal[0],  goal[1]] = 99
            return (cnt, results_arr)


In [9]:
def print_greedy_path(tmp_row, tmp_col):
    tmp_steps_result, tmp_array = follow_greedy_path(tmp_row, tmp_col)
    if tmp_steps_result > 0:
            print (tmp_array)
            print ('steps = ', tmp_steps_result ) 
    else:
        print ('Please train some more')            

In [10]:
list_of_states = [ [  initialize_states(r, c) for c in range(columns)] for r in range(rows)]

In [11]:
epsilon = 0.3
for _ in range(400):
    single_episode(start_state[0], start_state[1])
epsilon = 0.2
for _ in range(800):
    single_episode(start_state[0], start_state[1])    
epsilon = 0.1
for _ in range(8000):
    single_episode(start_state[0], start_state[1])     

In [12]:
print_greedy_path(start_state[0], start_state[1])

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  2.  0.  0.  0.  0. 99.  0.  0.]
 [ 0.  0.  0.  3.  4.  5.  6.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
steps =  7


In [ ]:
epsilon = 0.1
for cnt in range(10):
    print ('pass ' , cnt)
    print ('start ' , list_of_states[3][0])
    single_episode(start_state[0], start_state[1])
    steps_result, tmp_array = follow_greedy_path(start_state[0], start_state[1])
    if steps_result > 25:
        print ('steps = ', steps_result)
        print ( tmp_array)